###  Classification Targets

Target - the target is an ordinal variable indicating groups of income levels.

1 = extreme poverty \
2 = moderate poverty \
3 = vulnerable households \
4 = non vulnerable households


https://www.kaggle.com/competitions/costa-rican-household-poverty-prediction/data

### Setup

In [1]:
%pip install -r ../requirements.txt -q

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

In [3]:
DATA_DIR = '../data/'

TEST_CSV = DATA_DIR + 'test.csv'
TRAIN_CSV = DATA_DIR + 'train.csv'
TEST_CSV = DATA_DIR + 'test.csv'

TARGET_COLUMN = "Target"
ID_COLUMN = "Id"
HOUSE_HOLD_ID_COLUMN = "idhogar"

DEFAULT_RANDOM_STATE = 369
DEFAULT_TEST_SIZE = 0.3
DEFAULT_VALIDATION_SIZE = 0.3
DEFAULT_CROSS_VALIDATION = 4

In [4]:
train_data = pd.read_csv(TRAIN_CSV)

| column | description | values | 
| --- | --- | --- |
| bedrooms | number of bedrooms | 1-8 |
| overcrowding | # persons per room | 0.2 - 6 |
| meaneduc | meaneduc,average years of education for adults (18+) | 0-37 |



In [5]:
train_data

,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,...,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target
0,ID_279628684,190000.0,0,3,0,1,1,0,NaN,0,...,100,1849,1,100,0,1.000000,0.0000,100.0000,1849,4
1,ID_f29eb3ddd,135000.0,0,4,0,1,1,1,1.0,0,...,144,4489,1,144,0,1.000000,64.0000,144.0000,4489,4
2,ID_68de51c94,NaN,0,8,0,1,1,0,NaN,0,...,121,8464,1,0,0,0.250000,64.0000,121.0000,8464,4
3,ID_d671db89c,180000.0,0,5,0,1,1,1,1.0,0,...,81,289,16,121,4,1.777778,1.0000,121.0000,289,4
4,ID_d56d6f5f5,180000.0,0,5,0,1,1,1,1.0,0,...,121,1369,16,121,4,1.777778,1.0000,121.0000,1369,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9552,ID_d45ae367d,80000.0,0,6,0,1,1,0,NaN,0,...,81,2116,25,81,1,1.562500,0.0625,68.0625,2116,2
9553,ID_c94744e07,80000.0,0,6,0,1,1,0,NaN,0,...,0,4,25,81,1,1.562500,0.0625,68.0625,4,2
9554,ID_85fc658f8,80000.0,0,6,0,1,1,0,NaN,0,...,25,2500,25,81,1,1.562500,0.0625,68.0625,2500,2
9555,ID_ced540c61,80000.0,0,6,0,1,1,0,NaN,0,...,121,676,25,81,1,1.562500,0.0625,68.0625,676,2


In [96]:
sample = train_data.groupby(TARGET_COLUMN).apply(lambda x: x.sample(20))
sample[sample['dependency'] > 1][['overcrowding','meaneduc' ,'rooms', 'bedrooms', 'hogar_total' ,'edjefa','edjefe','dependency','SQBdependency', 'Target']].head()

overcrowding  meaneduc  rooms  bedrooms  hogar_total  edjefa  \
Target                                                                      
1      3394      1.666667       6.0      6         3            5     0.0   
       6213      4.000000       3.0      4         2            7     0.0   
       3004      1.500000       9.0      8         4            6     0.0   
       4491      2.500000       5.0      4         2            5     0.0   
       8441      2.000000       7.5      4         3            6     0.0   

             edjefe  dependency  SQBdependency  Target  
Target                                                  
1      3394     3.0         1.5           2.25       1  
       6213     3.0         2.5           6.25       1  
       3004    11.0         2.0           4.00       1  
       4491     1.0         1.5           2.25       1  
       8441     6.0         2.0           4.00       1

In [75]:
train_data['SQBdependency'].unique()

array([0.0000000e+00, 6.4000000e+01, 1.0000000e+00, 9.0000000e+00,
       2.5000000e-01, 6.2500000e-02, 4.0000000e+00, 4.4444448e-01,
       1.1111112e-01, 2.2500000e+00, 1.6000001e-01, 5.6250000e-01,
       1.5625000e+00, 4.0000003e-02, 6.2500000e+00, 1.4400001e+00,
       1.6000000e+01, 1.7777779e+00, 5.0625000e+00, 4.9382716e-02,
       2.5000000e+01, 6.9444442e-01, 6.4000005e-01, 3.6000000e+01,
       1.2250000e+01, 2.7777777e+00, 8.1632659e-02, 3.0625000e+00,
       5.1020408e-01, 2.7777780e-02, 3.6000001e-01])

In [74]:
# edjefe,years of education of male head of household
# edjefa,years of education of female head of household

# distinc values from edjefe and edjefa
train_data['dependency'].unique()


array(['no', '8', 'yes', '3', '.5', '.25', '2', '.66666669', '.33333334',
       '1.5', '.40000001', '.75', '1.25', '.2', '2.5', '1.2', '4',
       '1.3333334', '2.25', '.22222222', '5', '.83333331', '.80000001',
       '6', '3.5', '1.6666666', '.2857143', '1.75', '.71428573',
       '.16666667', '.60000002'], dtype=object)

In [81]:
# select dependency is yes
train_data[train_data['dependency'] == 'yes'][['dependency','SQBdependency']]

,dependency,SQBdependency
3,yes,1.0
4,yes,1.0
5,yes,1.0
6,yes,1.0
7,yes,1.0
...,...,...
9525,yes,1.0
9526,yes,1.0
9527,yes,1.0
9550,yes,1.0


In [91]:
# show columns that are not numeric
train_data.select_dtypes(exclude=['int64', 'float64']).columns

Index(['Id', 'idhogar'], dtype='object')

In [40]:
train_data['edjefa'].unique()

array(['no', '11', '4', '10', '9', '15', '7', '14', '13', '8', '17', '6',
       '5', '3', '16', '19', 'yes', '21', '12', '2', '20', '18'],
      dtype=object)

In [87]:
train_data['edjefa'] = train_data['edjefa'].replace({'no': 0, 'yes': 1}).astype(float)
train_data['edjefe'] = train_data['edjefe'].replace({'no': 0, 'yes': 1}).astype(float)
train_data['dependency'] = train_data['dependency'].replace({'no': 0, 'yes': 1}).astype(float)

In [49]:
train_data['edjefe'].unique()

array([10, 12,  0, 11,  9, 15,  4,  6,  8, 17,  7, 16, 14,  5, 21,  2, 19,
        1,  3, 18, 13, 20])

In [50]:
train_data['edjefa'].unique()

array([ 0, 11,  4, 10,  9, 15,  7, 14, 13,  8, 17,  6,  5,  3, 16, 19,  1,
       21, 12,  2, 20, 18])

In [48]:
train_data[['edjefa','edjefe']]

,edjefa,edjefe
0,0,10
1,0,12
2,11,0
3,0,11
4,0,11
...,...,...
9552,0,9
9553,0,9
9554,0,9
9555,0,9


In [71]:
# cumulative education is sum of educ level minus absolute value of educ level difference
train_data['cumulative_education'] = train_data['edjefa'] + train_data['edjefe'] - abs(train_data['edjefa'] - train_data['edjefe'])
train_data[(train_data['Target'] == 1 )][['overcrowding','meaneduc' ,'rooms', 'bedrooms', 'hogar_total' ,'edjefa','edjefe','cumulative_education', 'SQBdependency', 'Target']]


,overcrowding,meaneduc,rooms,bedrooms,hogar_total,edjefa,edjefe,cumulative_education,SQBdependency,Target
154,1.666667,9.5,5,3,5,0,8,0,2.25,1
155,1.666667,9.5,5,3,5,0,8,0,2.25,1
156,1.666667,9.5,5,3,5,0,8,0,2.25,1
157,1.666667,9.5,5,3,5,0,8,0,2.25,1
158,1.666667,9.5,5,3,5,0,8,0,2.25,1
...,...,...,...,...,...,...,...,...,...,...
9508,1.000000,4.0,4,2,2,0,3,0,0.00,1
9509,1.000000,4.0,4,2,2,0,3,0,0.00,1
9514,1.000000,5.0,3,2,2,5,0,0,1.00,1
9515,1.000000,5.0,3,2,2,5,0,0,1.00,1


In [70]:
train_data[(train_data['edjefa'] > 0) & (train_data['edjefe'] > 0)][['edjefa','edjefe']]

,edjefa,edjefe


In [67]:
train_data[(train_data['cumulative_education'] > 5) & (train_data['Target'] == 1 )][['overcrowding','meaneduc' ,'rooms', 'bedrooms', 'hogar_total' ,'edjefa','edjefe','cumulative_education', 'Target']]

,overcrowding,meaneduc,rooms,bedrooms,hogar_total,edjefa,edjefe,cumulative_education,Target


In [60]:
train_data[(train_data['edjefa'] > 1) & (train_data['edjefe'] > 5) & (train_data['Target'] == 1 )][['overcrowding','meaneduc' ,'rooms', 'bedrooms', 'hogar_total' ,'edjefa','edjefe', 'Target']]

,overcrowding,meaneduc,rooms,bedrooms,hogar_total,edjefa,edjefe,Target


In [14]:
train_data[(train_data['rooms'] > 5) & (train_data['Target'] == 1)]

,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,...,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target
237,ID_65ae0bd1f,NaN,0,6,0,1,1,0,NaN,0,...,64,7744,1,0,0,0.25,64.00,64.00,7744,1
322,ID_dab0d86a2,160000.0,0,6,0,1,1,0,NaN,0,...,36,900,36,0,16,4.00,4.00,36.00,900,1
987,ID_5152dd62c,NaN,0,7,0,1,1,0,NaN,1,...,9,121,9,0,4,2.56,4.00,36.00,121,1
988,ID_0a204866b,NaN,0,7,0,1,1,0,NaN,1,...,36,900,9,0,4,2.56,4.00,36.00,900,1
989,ID_0b4c30044,NaN,0,7,0,1,1,0,NaN,1,...,0,36,9,0,4,2.56,4.00,36.00,36,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9305,ID_ae71f8d9b,NaN,0,6,0,1,0,0,NaN,2,...,36,3136,36,36,4,4.00,0.25,12.25,3136,1
9306,ID_f1355afb3,NaN,0,6,0,1,0,0,NaN,2,...,0,16,36,36,4,4.00,0.25,12.25,16,1
9307,ID_ce6435b64,NaN,0,6,0,1,0,0,NaN,2,...,0,0,36,36,4,4.00,0.25,12.25,0,1
9308,ID_e0338b503,NaN,0,6,0,1,0,0,NaN,2,...,9,729,36,36,4,4.00,0.25,12.25,729,1


In [20]:
train_data[(train_data['rooms'] > 8) & (train_data['Target'] == 2)]

,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,...,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target
1660,ID_562104189,NaN,0,9,0,1,1,0,NaN,1,...,4,100,49,0,9,2.250000,0.562500,27.56250,100,2
1661,ID_5feecc29c,NaN,0,9,0,1,1,0,NaN,1,...,36,841,49,0,9,2.250000,0.562500,27.56250,841,2
1662,ID_e0b20fae2,NaN,0,9,0,1,1,0,NaN,1,...,25,625,49,0,9,2.250000,0.562500,27.56250,625,2
1663,ID_0b37aa51b,NaN,0,9,0,1,1,0,NaN,1,...,1,49,49,0,9,2.250000,0.562500,27.56250,49,2
1664,ID_064a73aeb,NaN,0,9,0,1,1,0,NaN,1,...,16,441,49,0,9,2.250000,0.562500,27.56250,441,2
1665,ID_6c22df966,NaN,0,9,0,1,1,0,NaN,1,...,36,2025,49,0,9,2.250000,0.562500,27.56250,2025,2
1666,ID_f3715e469,NaN,0,9,0,1,1,0,NaN,1,...,0,16,49,0,9,2.250000,0.562500,27.56250,16,2
4135,ID_904781cfa,NaN,0,9,0,1,1,0,NaN,1,...,16,5184,25,0,9,0.694444,16.000000,64.00000,5184,2
4136,ID_82a6f208b,NaN,0,9,0,1,1,0,NaN,1,...,0,0,25,0,9,0.694444,16.000000,64.00000,0,2
4137,ID_943f4445b,NaN,0,9,0,1,1,0,NaN,1,...,36,1600,25,0,9,0.694444,16.000000,64.00000,1600,2


In [18]:
train_data[(train_data['rooms'] > 5) & (train_data['Target'] == 3)]

,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,...,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target
406,ID_7d5b483e6,180000.0,0,7,0,1,1,0,NaN,0,...,121,1521,49,0,25,1.960000,6.2500,72.250000,1521,3
407,ID_16669b3ac,180000.0,0,7,0,1,1,0,NaN,0,...,0,25,49,0,25,1.960000,6.2500,72.250000,25,3
408,ID_14b967c3b,180000.0,0,7,0,1,1,0,NaN,0,...,16,144,49,0,25,1.960000,6.2500,72.250000,144,3
409,ID_aa0e92ddc,180000.0,0,7,0,1,1,0,NaN,0,...,0,49,49,0,25,1.960000,6.2500,72.250000,49,3
412,ID_6895c3ee3,180000.0,0,7,0,1,1,0,NaN,0,...,36,2025,49,0,25,1.960000,6.2500,72.250000,2025,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9453,ID_2f5ad837a,NaN,0,6,0,1,1,0,NaN,0,...,4,64,49,0,4,3.062500,0.5625,12.959999,64,3
9454,ID_d74271567,NaN,0,6,0,1,1,0,NaN,0,...,36,1681,49,0,4,3.062500,0.5625,12.959999,1681,3
9455,ID_dc7068500,NaN,0,6,0,1,1,0,NaN,0,...,0,3481,49,0,4,3.062500,0.5625,12.959999,3481,3
9456,ID_cd592016b,NaN,0,6,0,1,1,0,NaN,0,...,36,256,49,0,4,3.062500,0.5625,12.959999,256,3
